#Fine-tuning T5 from the Huggingface Library Simple Transformers

This notebook trains T5 on OpSpam and tests on Yelp.

In [ ]:
!pip install simpletransformers
import pandas as pd
from simpletransformers.t5 import T5Model
from pprint import pprint
import logging
# Making sure the environment is set up correctly for anyone running this notebook
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import datetime as datetime
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import sklearn
from google.colab import drive
from tensorflow import keras
import re

     |████████████████████████████████| 215kB 15.8MB/s 
     |████████████████████████████████| 81kB 4.1MB/s 
     |████████████████████████████████| 1.8MB 28.1MB/s 
     |████████████████████████████████| 8.2MB 58.4MB/s 
     |████████████████████████████████| 225kB 54.0MB/s 
     |████████████████████████████████| 1.2MB 50.0MB/s 
     |████████████████████████████████| 51kB 9.6MB/s 
     |████████████████████████████████| 122kB 59.0MB/s 
     |████████████████████████████████| 3.3MB 50.2MB/s 
     |████████████████████████████████| 2.1MB 47.7MB/s 
     |████████████████████████████████| 163kB 61.6MB/s 
     |████████████████████████████████| 133kB 57.6MB/s 
     |████████████████████████████████| 102kB 15.4MB/s 
     |████████████████████████████████| 112kB 57.0MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
     |████████████████████████████████| 4.2MB 50.2MB/s 
     |████████████████████████████████| 245kB 56.3MB/s 
     |████████████████████████████████| 112kB 33.6MB/

In [ ]:
# Settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

**Getting the Data**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


**Load data**

In [ ]:
hotel_test= pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/hotel_testii.pkl')
hotel_train=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/hotel_trainii.pkl')
yelp=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/yelp.pkl')

In [ ]:
hotel_test.head()

,input_text,target_text,prefix
0,Although the Millennium Knickerbocker Hotel Ch...,1,binary classification
1,We stayed in the Conrad for 4 nights just befo...,0,binary classification
2,The InterContinental Chicago is conveniently l...,1,binary classification
3,The Fairmont Chicago Millennium Park Hotel was...,1,binary classification
4,took a weekend trip with my wife. got a great ...,0,binary classification


**Train the model**

In [ ]:
# Model arguments. Be careful to set arguments that allow the use of custom metrics.  
# https://simpletransformers.ai/docs/t5-model/
# We use convention-based defaults for the training batch size, and the number of epochs
# Using a small # of epochs has a regularization effect (i.e. balances wanting to fit the training data well, and finding estimates
# for parameters that generalize well 

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 320, 
    "train_batch_size": 32, #convention is 32; small batch sizes are noisy and offer a regularizing effect; the # of observations in the training dataset must be divisible by the train_batch_size
    "num_train_epochs": 3,
    "save_eval_checkpoints": True, 
    "save_model_every_epoch": True,
    # "silent": True,
    "evaluate_generated_text": True,
    "evaluate_during_training": True, #[open] try setting to false for more epochs? for some reason, when i get to epoch 4 i get thrown an error -- dictionary given where numpy or tensor object expected
    "evaluate_during_training_verbose": True, #try setting to false for more epochs?
    #"num_workers": 1, doesn't fix memory problem...
}


In [ ]:
# Custom metrics function

def metrics_fn(l, p):

    # Change into integer types 
    l_int = np.array(l).astype(int)
    p_int = np.array(p).astype(int)

    eval_accuracy=sklearn.metrics.accuracy_score(l, p)
    f1_score =sklearn.metrics.f1_score(l, p, labels=['0', '1'], pos_label = '1')
    auc = sklearn.metrics.roc_auc_score(l_int, p_int)
    precision = sklearn.metrics.precision_score(l, p, pos_label = '1')
    recall = sklearn.metrics.recall_score(l, p, labels=['0', '1'], pos_label = '1')
    cm = sklearn.metrics.confusion_matrix(l, p, labels=['0', '1'])
    return {'accuracy': eval_accuracy,
            'f1_score': f1_score,
            'precision': precision,
            'recall': recall,
            'auc': auc,
            'confusion matrix': cm}
    #return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

In [ ]:

# Instantiate the model #Hard to get GPU on colab

model = T5Model("t5","t5-small", args=model_args, use_cuda = True)
# Train the model
# Common errors returned:
##                         mmap: cannnot allocate memory. In this case, "Restart runtime", and "Run all".
model.train_model(hotel_train, output_dir = "/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_hotel", eval_data=hotel_test, metrics = metrics_fn)


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

  0%|          | 0/1280 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Generating outputs:   0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/320 [00:00<?, ?it/s]

(120,
 {'eval_loss': [0.2291175454854965, 0.1936310103163123, 0.20781896403059363],
  'global_step': [40, 80, 120],
  'metrics': [{'accuracy': 0.671875,
    'auc': 0.671875,
    'confusion matrix': array([[ 66,  94],
           [ 11, 149]]),
    'f1_score': 0.739454094292804,
    'precision': 0.6131687242798354,
    'recall': 0.93125},
   {'accuracy': 0.78125, 'auc': 0.78125, 'confusion matrix': array([[105,  55],
           [ 15, 145]]), 'f1_score': 0.8055555555555555, 'precision': 0.725, 'recall': 0.90625},
   {'accuracy': 0.775, 'auc': 0.775, 'confusion matrix': array([[153,   7],
           [ 65,  95]]), 'f1_score': 0.7251908396946565, 'precision': 0.9313725490196079, 'recall': 0.59375}],
  'train_loss': [0.2729734480381012,
   0.18359200656414032,
   0.17141935229301453]})

**Evaluate**

In [ ]:
# Load model
# model = T5Model("t5", "/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_ts/checkpoint-852-epoch-3")

In [ ]:
# Print out model metrics on training data
'''
result_yelp_train = model.eval_model(yelp_old_train, metrics = metrics_fn)
out=pd.DataFrame([result_yelp_train['metrics']])
out.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_ts/m_training.csv")
print(out)
'''

'\nresult_yelp_train = model.eval_model(yelp_old_train, metrics = metrics_fn)\nout=pd.DataFrame([result_yelp_train[\'metrics\']])\nout.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_ts/m_training.csv")\nprint(out)\n'

In [ ]:
# Print out model metrics on testing data

result_test = model.eval_model(hotel_test, metrics = metrics_fn)
out = pd.DataFrame([result_test['metrics']])
out.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_hotel/m_testing.csv")
print(out)

  0%|          | 0/320 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/320 [00:00<?, ?it/s]

   accuracy  f1_score  precision   recall    auc      confusion matrix
0     0.775  0.725191   0.931373  0.59375  0.775  [[153, 7], [65, 95]]
